# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import calendar
from sklearn.preprocessing import LabelEncoder

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


We remove `year` part from `issue_d` and `next_pymnt_d` columns as year is same for all the rows which is `2019`

In [5]:
df['issue_d'] = df['issue_d'].str[0:3]
df['next_pymnt_d'] = df['next_pymnt_d'].str[0:3]

## Integer Encoding

### Manual Integer Encoding
Perform a manual integer encoding of the Month column, use a dictionary to map months names with their corresponding numerical value.

In [6]:
# Months dictionary
months_num = {name: num for num, name in enumerate(calendar.month_abbr) if num}
months_num

{'Jan': 1,
 'Feb': 2,
 'Mar': 3,
 'Apr': 4,
 'May': 5,
 'Jun': 6,
 'Jul': 7,
 'Aug': 8,
 'Sep': 9,
 'Oct': 10,
 'Nov': 11,
 'Dec': 12}

In [7]:
# Encode month name
df["issue_d"] = df["issue_d"].apply(lambda x: months_num[x])
df["next_pymnt_d"] = df["next_pymnt_d"].apply(lambda x: months_num[x])

In [8]:
# Create the LabelEncoder instance
label_encoder = LabelEncoder()

# Fitting and encoding the columns with the LabelEncoder
for col in df[
    ['verification_status', 
     'pymnt_plan', 
     'initial_list_status', 
     'application_type', 
     'hardship_flag', 
     'debt_settlement_flag']]:
    label_encoder.fit(df[col])
    df[col] = label_encoder.transform(df[col])

### Encoding Data using get_dummies()
Perform a binary encoding on the `home_ownership` column using the Pandas get_dummies() function.

In [9]:
# Encoding the `home ownership` column
df = pd.get_dummies(df, columns=['home_ownership'])
df.head()

,loan_amnt,int_rate,installment,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,delinq_2yrs,...,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT
0,10500.0,0.1719,375.35,66000.0,1,3,low_risk,0,27.24,0.0,...,65687.0,38199.0,2000.0,61987.0,0,0,0,0,0,1
1,25000.0,0.2000,929.09,105000.0,2,3,low_risk,0,20.23,0.0,...,271427.0,60641.0,41200.0,49197.0,0,0,0,1,0,0
2,20000.0,0.2000,529.88,56000.0,2,3,low_risk,0,24.26,0.0,...,60644.0,45684.0,7500.0,43144.0,0,0,0,1,0,0
3,10000.0,0.1640,353.55,92000.0,2,3,low_risk,0,31.44,0.0,...,99506.0,68784.0,19700.0,76506.0,0,0,0,0,0,1
4,22000.0,0.1474,520.39,52000.0,0,3,low_risk,0,18.76,0.0,...,219750.0,25919.0,27600.0,20000.0,0,0,0,1,0,0


### Save the Preprocessed File

Finally, save the preprocessed file as df_encoded.csv for forthcoming usage.

In [10]:
# Save the file for forthcoming usage
file_path = Path("Resources/df_encoded.csv")
df.to_csv(file_path, index=False)
df.head()

,loan_amnt,int_rate,installment,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,delinq_2yrs,...,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT
0,10500.0,0.1719,375.35,66000.0,1,3,low_risk,0,27.24,0.0,...,65687.0,38199.0,2000.0,61987.0,0,0,0,0,0,1
1,25000.0,0.2000,929.09,105000.0,2,3,low_risk,0,20.23,0.0,...,271427.0,60641.0,41200.0,49197.0,0,0,0,1,0,0
2,20000.0,0.2000,529.88,56000.0,2,3,low_risk,0,24.26,0.0,...,60644.0,45684.0,7500.0,43144.0,0,0,0,1,0,0
3,10000.0,0.1640,353.55,92000.0,2,3,low_risk,0,31.44,0.0,...,99506.0,68784.0,19700.0,76506.0,0,0,0,0,0,1
4,22000.0,0.1474,520.39,52000.0,0,3,low_risk,0,18.76,0.0,...,219750.0,25919.0,27600.0,20000.0,0,0,0,1,0,0


## Split the Data into Training and Testing

In [11]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y = df["loan_status"]
X.head()

,loan_amnt,int_rate,installment,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT
0,10500.0,0.1719,375.35,66000.0,1,3,0,27.24,0.0,0.0,...,65687.0,38199.0,2000.0,61987.0,0,0,0,0,0,1
1,25000.0,0.2000,929.09,105000.0,2,3,0,20.23,0.0,0.0,...,271427.0,60641.0,41200.0,49197.0,0,0,0,1,0,0
2,20000.0,0.2000,529.88,56000.0,2,3,0,24.26,0.0,0.0,...,60644.0,45684.0,7500.0,43144.0,0,0,0,1,0,0
3,10000.0,0.1640,353.55,92000.0,2,3,0,31.44,0.0,1.0,...,99506.0,68784.0,19700.0,76506.0,0,0,0,0,0,1
4,22000.0,0.1474,520.39,52000.0,0,3,0,18.76,0.0,1.0,...,219750.0,25919.0,27600.0,20000.0,0,0,0,1,0,0


In [12]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,...,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,0.669994,1.726172,0.0,21.778153,0.217766,0.497697,...,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0,0.009285,0.526309,0.106747,0.357659
std,10277.348590,0.048130,288.062432,1.155800e+05,0.719105,0.743862,0.0,20.199244,0.718367,0.758122,...,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0,0.095914,0.499311,0.308793,0.479314
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,1.000000,0.0,0.000000,0.000000,0.000000,...,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,0.000000,1.000000,0.0,13.890000,0.000000,0.000000,...,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0,0.000000,0.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,1.000000,2.000000,0.0,19.760000,0.000000,0.000000,...,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0,0.000000,1.000000,0.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,1.000000,2.000000,0.0,26.660000,0.000000,1.000000,...,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0,0.000000,1.000000,0.000000,1.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,2.000000,3.000000,0.0,999.000000,18.000000,5.000000,...,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0,1.000000,1.000000,1.000000,1.000000


In [13]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [14]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

(51612, 88)

In [15]:
y_train.shape

(51612,)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [16]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [17]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [18]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled

array([[-0.64978133, -0.30327574, -0.52669276, ...,  0.94255244,
        -0.34381364, -0.74237524],
       [ 0.42378352, -0.42616892,  0.71269346, ..., -1.06094892,
        -0.34381364,  1.34702769],
       [ 1.79013878, -0.82400957,  2.19335392, ..., -1.06094892,
         2.90855246, -0.74237524],
       ...,
       [ 0.81417074, -0.36368086,  1.17925683, ...,  0.94255244,
        -0.34381364, -0.74237524],
       [-0.25939411, -0.42616892, -0.08220905, ...,  0.94255244,
        -0.34381364, -0.74237524],
       [-0.16179731, -1.196855  , -0.05849257, ...,  0.94255244,
        -0.34381364, -0.74237524]])

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [19]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf_model = brf_model.fit(X_train_scaled, y_train)

In [20]:
# Calculated the balanced accuracy score
brf_predictions = brf_model.predict(X_test_scaled)
brf_acc_score = balanced_accuracy_score(y_test, brf_predictions)
print(f"The balanced accuracy score is {brf_acc_score}")

The balanced accuracy score is 0.7627854929878208


In [21]:
# Display the confusion matrix
brf_cm = confusion_matrix(y_test, brf_predictions)
brf_cm_df = pd.DataFrame(
    brf_cm, index=["Actual high_risk", "Actual low_risk "], columns=["Predicted high_risk", "Predicted low_risk"]
)
brf_cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,55,32
Actual low_risk,1825,15293


In [22]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, brf_predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.63      0.89      0.06      0.75      0.55        87
   low_risk       1.00      0.89      0.63      0.94      0.75      0.58     17118

avg / total       0.99      0.89      0.63      0.94      0.75      0.58     17205



In [23]:
# List the features sorted in descending order by feature importance
feature_importance = pd.DataFrame(brf_model.feature_importances_,
                                   index = X.columns,
                                    columns=['Feature Importance']).sort_values('Feature Importance', ascending = False)
feature_importance[:10]

,Feature Importance
total_rec_prncp,0.078364
last_pymnt_amnt,0.071532
total_rec_int,0.059693
total_pymnt_inv,0.056803
total_pymnt,0.048883
issue_d,0.029705
int_rate,0.025360
dti,0.022306
out_prncp,0.017927
annual_inc,0.017248


### Easy Ensemble Classifier

In [24]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
eec_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec_model.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [25]:
# Calculated the balanced accuracy score
eec_predictions = eec_model.predict(X_test_scaled)
eec_acc_score = balanced_accuracy_score(y_test, eec_predictions)
print(f"The balanced accuracy score is {eec_acc_score}")

The balanced accuracy score is 0.9319339191252604


In [26]:
# Display the confusion matrix
eec_cm = confusion_matrix(y_test, eec_predictions)
eec_cm_df = pd.DataFrame(
    eec_cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"]
)
eec_cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,80,7
Actual low_risk,953,16165


In [27]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, eec_predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      0.92      0.94      0.14      0.93      0.87        87
   low_risk       1.00      0.94      0.92      0.97      0.93      0.87     17118

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205



## Observations

We resample our training data by using the different algorithms and we get the following performance score of the models e.g. balanced accuracy score, recall score and geometric mean score:

- Train our data by using Balanced Random Forest Classifier algorithm, and the model performance:
    * balanced accuracy score 0.76
    * high risk recall score 0.63 and low risk recall score 0.89 
    * geometric mean score 0.75
    * high risk f1 score 0.06 and low risk f1 score 0.94
    

- Train our data by using Easy Ensemble Classifier algorithm, and the model performance:
    * balanced accuracy scores are 0.93
    * high risk recall score 0.92 and low risk recall score 0.94
    * geometric mean scores are 0.93
    * high risk f1 score 0.14 and low risk f1 score 0.97

### Final Questions

**Which model had the best balanced accuracy score?**

The model which use the easy ensemble classifier algorithm has the best balanced accuracy score e.g. 0.93

**Which model had the best recall score?**

The model which use the easy ensemble classifier algorithm has the best avg/total recall score e.g. 0.94

**Which model had the best geometric mean score?**

The model which use the easy ensemble classifier algorithm has the best geometric mean score e.g. 0.93

**What are the top three features?**

The top three features are as following:
    1. total_rec_prncp(Principal received to date)
    2. last_pymnt_amnt(Last total payment amount received)
    3. total_rec_int(Interest received to date)